<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb SynapticPlasticity.do.txt  -->

## Short term plasticity
Facilitation and depression can both be modeled as 
presynaptic processes that modify the probability of transmitter release. For both facilitation and depression, 
the release probability after a long period of presynaptic silence is $P_{rel} = P_0$. 
Activity at the synapse causes $P_{rel}$ to increase in the case of facilitation and to decrease for depression. 
Between presynaptic action potentials, the release probability decays exponentially back to its 'resting' value $P_0$,

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\tau_P \frac{dP_{rel}}{dt} = P_0 - P_{rel}
\label{_auto1} \tag{1}
\end{equation}
$$

The parameter $\tau_P$ controls the rate at which the release probability decays to $P_0$. 
The models of facilitation and depression differ in how the release probability is changed by presynaptic activity. 
In the case of facilitation, $P_{rel}$ is augmented by making the replacement 
$P_{rel} \rightarrow P_{rel} + f_F (1 - P_{rel})$ immediately after a presynaptic action potential. 
The parameter $f_F$ (with $0 \leq f_F \leq 1$) controls the degree of facilitation, and the factor $(1 - P_{rel})$ 
prevents the release probability from growing larger than one. To model depression, the release probability is reduced 
after a presynaptic action potential by making the replacement $P_{rel} \rightarrow f_D P_{rel}$. 
In this case, the parameter $f_D$ (with $0 \leq f_D \leq 1$) controls the amount of depression, and the 
factor $P_{rel}$ prevents the release probability from becoming negative.

## Exercise 1: Facilitation and depression in Brian2

In this exercise, you will implement a model of facilitation and depression in Brian2.

**a)**
Define the model

In [1]:
from brian2 import *
start_scope()

# Simulation parameters
duration = 0.5*second
dt = 0.001*ms
defaultclock.dt = dt

# Neuron parameters
u_rest = -65*mV  # mV
g_l = 10*uS
C_m = 200*pF

# Synapse parameters
E_s = 0*mV
tau_s = 1*ms
tau_p = 100*ms
g_s_bar = 10*nS
ff = 0.5  # factor for synaptic facilitation
# fd = 0.5  # factor for synaptic depression
P0 = 0.1
Ps_max = 1

eqs = '''
du/dt = code here / C_m : volt
g : siemens
'''
model = '''
dPrel/dt = code here: 1 (clock-driven)
dPs/dt = code here : 1 (clock-driven)
g_post = code here : siemens (summed)
'''
neurons = NeuronGroup(1, eqs, method='euler')
neurons.u = u_rest
times = arange(10,200,20)*ms  # Set up your spike times here
indices = array([0]*len(times))
spikes = SpikeGeneratorGroup(1, indices, times)

<!-- --- begin solution of exercise --- -->
**Solution.**

In [2]:
eqs = '''
du/dt = (g_l*(u_rest - u) - g*u_rest) / C_m : volt
g : siemens
'''
model = '''
dPrel/dt = (P0 - Prel)/tau_p : 1 (clock-driven)
dPs/dt = - Ps / tau_s : 1 (clock-driven)
g_post = g_s_bar*Prel*Ps : siemens (summed)
'''

<!-- --- end solution of exercise --- -->

**b)**
Define the synapses

In [3]:
synapse = Synapses(spikes, neurons, model, on_pre='code here')
# synapse = Synapses(spikes, neurons, model, on_pre='code here')

<!-- --- begin solution of exercise --- -->
**Solution.**

In [4]:
synapse = Synapses(spikes, neurons, model, on_pre='Prel += ff*(1-Prel); Ps += Ps_max*(1-Ps)')
# synapse = Synapses(spikes, neurons, model, on_pre='Prel = fd*Prel; Ps += Ps_max*(1-Ps)')

<!-- --- end solution of exercise --- -->

In [5]:
synapse.connect()
synapse.Prel = 0.1

# Monitor membrane potential
monitor = StateMonitor(neurons, ['u', 'g'], record=True)
monitor_s = StateMonitor(synapse, ['Ps', 'Prel'], record=True)

# Run simulation
run(duration)

figure(figsize=(9, 4))
plot(monitor.t/ms, monitor.g[0]/nS)
xlabel('Time (ms)')
ylabel('Total conductance (nS)')
figure(figsize=(9, 4))
plot(monitor.t/ms, monitor.u[0]/mV)
xlabel('Time (ms)')
ylabel('Membrane potential (mV)')

figure(figsize=(9, 4))
plot(monitor.t/ms, monitor_s.Prel[0])
xlabel('Time (ms)')
ylabel('Release probability')
figure(figsize=(9, 4))
plot(monitor.t/ms, monitor_s.Ps[0])
xlabel('Time (ms)')
ylabel('Post-synaptic Probability')

## Plasticity rules

Here we explore activity-dependent synaptic plasticity, focusing on Hebbian type and its augmentation with more 
global synaptic modifications. Non-Hebbian synaptic plasticity, which modifies synaptic strengths based solely on 
pre- or postsynaptic firing, is emphasized as an important factor in homeostatic, developmental, and learning processes. 
Furthermore, we also explore the influence of activity on intrinsic excitability and response properties of neurons, and 
the interplay of intrinsic and synaptic plasticity.

Hebbian plasticity leads to an increasing synaptic strength, but without constraints it can result in uncontrolled 
growth of synaptic strengths. An upper limit on synaptic weight can serve as a control measure, supported by LTP experiments. 
To this end one can impose a saturation constraint ensuring that all excitatory synaptic weights lie between zero 
and a maximum constant value, $w_{max}$. 

Adequate synaptic development typically requires competition between different synapses, prompting some to weaken 
when others strengthen. We discuss several synaptic plasticity rules introducing such competition.

Synaptic plasticity rules take the form of differential equations, where the rate of change of synaptic weights 
depends on the pre- and postsynaptic activity. In the models studied, neuronal activity is represented by a continuous 
variable, $u$ and $v$ for presynaptic and postsynaptic activity respectively. 

In the initial segment, we explore the application of unsupervised learning to a single postsynaptic neuron that is 
influenced by $\text{Nu}$ presynaptic inputs. 
The activities of these inputs are denoted by $u_b$ for $b = 1, 2, \ldots, \text{Nu}$ or, collectively, 
by the vector $\mathbf{u}$. As the learning is unsupervised, the postsynaptic activity $\mathbf{v}$ is 
directly prompted by the presynaptic activity $\mathbf{u}$, not by an external entity.

We utilize a linear variant of the firing-rate model, which can be written as:

<!-- Equation labels as ordinary links -->
<div id="eq:rate_model"></div>

$$
\begin{equation}
\tau_r \frac{dv}{dt} = -v + \mathbf{w} \cdot \mathbf{u} = -v + \sum_{b=1}^{\text{Nu}} w_b u_b
\label{eq:rate_model} \tag{2}
\end{equation}
$$

Here, $\tau_r$ is a time constant which manages the dynamics of the firing rate response. 
Moreover, $w_b$ denotes the synaptic weight, describing the strength of the synapse from the 
presynaptic neuron $b$ to the postsynaptic neuron, and $\mathbf{w}$ is the vector comprising all $\text{Nu}$ synaptic weights.
These synaptic weights can be positive, indicating excitation, or negative, indicating inhibition. 
Equation ([2](#eq:rate_model)) does not encompass any non-linear dependence of the firing rate on the total synaptic input, 
including rectification.

Adopting such a linear firing-rate model significantly streamlines the analysis of synaptic plasticity. 
The limitation to non-negative $\mathbf{v}$ will be either imposed manually, or sometimes overlooked to simplify 
the analysis.

The mechanisms of synaptic plasticity are usually much slower than the dynamics outlined by equation ([2](#eq:rate_model)). 
Furthermore, if the stimuli are introduced slowly enough to enable the network to reach its steady-state activity 
during training, we can substitute the dynamic equation ([2](#eq:rate_model)) by:

<!-- Equation labels as ordinary links -->
<div id="eq:rate_model_ss"></div>

$$
\begin{equation}
v = \mathbf{w} \cdot \mathbf{u}
\label{eq:rate_model_ss} \tag{3}
\end{equation}
$$

This equation instantaneously sets $v$ to the asymptotic, steady-state value determined by equation ([2](#eq:rate_model)). 
This is the primary equation we utilize in our analysis of synaptic plasticity in unsupervised learning.

Synaptic modifications are included in the model by designating how the vector $\mathbf{w}$ alters as a function of 
the pre- and postsynaptic levels of activity.

## The Simplest Plasticity Rule: Basic Hebb Rule

The simplest form of plasticity rule that is consistent with Hebb's conjecture is given by the following equation:

<!-- Equation labels as ordinary links -->
<div id="eq:basic_hebb"></div>

$$
\begin{equation}
\tau_w \frac{\mathrm{d}\mathbf{w}}{\mathrm{d}t} = v\mathbf{u}
\label{eq:basic_hebb} \tag{4}
\end{equation}
$$

This implies that the simultaneous firing of pre- and postsynaptic neurons enhances synaptic strength, 
which we refer to as the basic Hebb rule. 
If the activity variables symbolize firing rates, the right-hand side of this equation could be viewed as a 
measure of the likelihood of the pre- and postsynaptic neurons both firing spikes during a small time interval. 
Here, $\tau_w$ is a time constant that regulates the rate of weight changes.

Synaptic plasticity is generally modeled as a slow process, where the input pattern $u$ take on a variety of values. 
To calculate the weight changes induced by a series of input patterns we average over all 
different input patterns and calculate the weight changes induced by this average. 
As long as the synaptic weights change slowly enough, this averaging method provides a 
good approximation of the weight changes produced by the set of input patterns.

We use angle brackets $\langle \rangle$ to denote averages over the ensemble of input patterns presented during 
training. 
The Hebb rule of Eq. ([4](#eq:basic_hebb)), when averaged over the inputs used during training, becomes:

<!-- Equation labels as ordinary links -->
<div id="eq:basic_hebb_averaged"></div>

$$
\begin{equation}
\tau_w \frac{\mathrm{d}\mathbf{w}}{\mathrm{d}t} = \langle v\mathbf{u} \rangle
\label{eq:basic_hebb_averaged} \tag{5}
\end{equation}
$$

We replace $v$ by $\mathbf{w} \cdot \mathbf{u}$, and introduce $\mathbf{Q}$ as the input correlation 
matrix defined by $\mathbf{Q} = \langle \mathbf{u}\mathbf{u} \rangle$. 
With this we can rewrite the averaged plasticity rule Eq. ([5](#eq:basic_hebb_averaged)) as:

<!-- Equation labels as ordinary links -->
<div id="eq:basic_hebb_averaged_2"></div>

$$
\begin{equation}
\tau_w \frac{\mathrm{d}\mathbf{w}}{\mathrm{d}t} &= \mathbf{Q} \cdot \mathbf{w}
\label{eq:basic_hebb_averaged_2} \tag{6}
\end{equation}
$$

Equation ([6](#eq:basic_hebb_averaged_2)) is referred to as a correlation-based plasticity 
rule due to the presence of the input correlation matrix.

The Hebb rule, tends to cause unbounded weight growth because it lacks an upper limit and 
fails to generate competition among different synapses. 
This can be demonstrated by examining the square of the weight vector length and its 
change over time yielding $\tau_w \frac{d|\mathbf{w}|^2}{dt} = 2v^2$, which is always positive 
and the length of the weight vector continuously grows. A method to control this growth is by 
introducing an upper saturation constraint and a lower limit if the activity variables can 
be negative.

## The Covariance Rule

The Covariance Rule is another synaptic plasticity rule which models the way synaptic 
strength can increase or decrease depending on the level of postsynaptic activity. 
It is represented by the formula:

$$
\tau_w \frac{\mathrm{d}\mathbf{w}}{\mathrm{d}t} = (v - \theta_v)\mathbf{u}
$$

Here, $\theta_v$ is a threshold that determines the level of postsynaptic activity 
above which long-term depression switches to long-term potentiation. 
Alternatively, we can impose the threshold on the input activity instead of the 
output activity:

$$
\tau_w \frac{\mathrm{d}\mathbf{w}}{\mathrm{d}t} = v(\mathbf{u} - \mathbf{\theta_u})
$$

In this case, $\mathbf{\theta_u}$ is a vector of thresholds that determines the levels of 
presynaptic activities above which LTD switches to LTP. 
The thresholds are usually set to the average value of the corresponding variable 
over the training period. These equations are known as covariance rules because of the 
presence of the covariance matrix in the averaged form of the plasticity rule:

$$
\tau_w \frac{\mathrm{d}\mathbf{w}}{\mathrm{d}t} = \mathbf{C} \cdot \mathbf{w}
$$

Here, $\mathbf{C}$ is the covariance matrix defined as 
$\mathbf{C} = \langle \mathbf{u} \mathbf{u} \rangle - \langle \mathbf{u} \rangle \langle \mathbf{u} \rangle$.

Although the covariance rules include long-term depression, they are unstable due to 
the same positive feedback that makes the basic Hebb rule unstable.

## The BCM Rule
Finally, there's the BCM rule which requires both pre- and postsynaptic activity to 
change a synaptic weight. It's formulated as:

$$
\tau_w \frac{\mathrm{d}\mathbf{w}}{\mathrm{d}t} = v\mathbf{u}(v - \theta_v)
$$

In this case, $\theta_v$ is a variable threshold on the postsynaptic activity that determines 
whether synapses are strengthened or weakened. 
The critical condition for stability is that $\theta_v$ must grow faster than $v$ 
if the output activity grows large.
The stability of the BCM rule can be achieved, in one implementation, 
where the threshold follows the square of the postsynaptic activity:

$$
\tau_\theta \frac{d\theta_v}{dt} = v^2 - \theta_v
$$

Here, $\tau_\theta$ sets the time scale for the modification of the threshold. 
The BCM rule implements competition between synapses because strengthening some 
synapses increases the postsynaptic firing rate, which raises the threshold and 
makes it more difficult for other synapses to be strengthened or remain at their 
current strengths.

## Exercise 2: Implement the BCM rule

Plot the evolution of weights and theta over time

<!-- --- begin solution of exercise --- -->
**Solution.**

In [6]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
def BCM(pre_syn_activity, post_syn_activity, weights, tau_w=0.1, tau_theta=0.1, learning_rate=0.01):
    # Initialize threshold
    theta_v = 0.5
    
    num_iterations = len(pre_syn_activity)
    
    # We keep track of the evolution of weights and theta
    weight_evolution = np.zeros(num_iterations)
    theta_evolution = np.zeros(num_iterations)
    
    for t in range(num_iterations):
        # Calculate the weight update according to the BCM rule
        dw = post_syn_activity[t] * pre_syn_activity[t] * (post_syn_activity[t] - theta_v)
        
        # Update the weight
        weights += learning_rate * dw / tau_w
        
        # Update the threshold
        theta_v += learning_rate * (post_syn_activity[t]**2 - theta_v) / tau_theta
        
        # Store the weights and theta values
        weight_evolution[t] = weights
        theta_evolution[t] = theta_v
    
    return weights, weight_evolution, theta_evolution

# Let's test the BCM rule with some data
pre_syn_activity = np.random.rand(1000)
post_syn_activity = np.random.rand(1000)
initial_weights = 0.5

final_weights, weight_evolution, theta_evolution = BCM(pre_syn_activity, post_syn_activity, initial_weights)

plt.figure()
plt.plot(weight_evolution)
plt.title('Evolution of weights')

plt.figure()
plt.plot(theta_evolution)
plt.title('Evolution of theta')
plt.show()

Here, pre_syn_activity and post_syn_activity represent the pre- and post-synaptic activities, respectively, and 
weights represent the synaptic weights. The function BCM updates the weights and the threshold (theta_v) 
according to the BCM rule over the course of several iterations (set by the length of pre_syn_activity). 
The learning rate and the time constants for the weights and the threshold can be set by learning_rate, 
tau_w, and tau_theta, respectively. The function returns the final weights and the evolution of weights and 
threshold over time.

<!-- --- end solution of exercise --- -->

## Weight Normalization
The BCM rule achieves stabilization of Hebbian plasticity by employing a dynamic 
threshold that reduces synaptic weights when the postsynaptic neuron activity is 
excessively high, essentially using postsynaptic activity as an index of synaptic weight 
strengths. An alternative stabilization method introduces weight-dependent terms, leading 
to weight normalization, built on the assumption that postsynaptic neurons can only 
sustain a constant total synaptic weight, indicating that weight increases must be 
compensated by decreases elsewhere.

Synaptic weight normalization involves applying a global constraint. Two types of 
constraints are typically employed. For non-negative synaptic weights, their growth can 
be bounded by maintaining a constant sum of all the weights of the synapses onto a 
given postsynaptic neuron. Alternatively, for weights that can be both positive or 
negative, one can constrain the sum of the squares of the weights rather than their 
linear sum. In both cases, the constraint can be enforced either strictly, requiring 
it to be met always during training, or dynamically, needing it to be met asymptotically 
at training's end.

A constraint on the sum of the squares of the synaptic weights can be dynamically enforced 
using a modification of the basic Hebb rule, known as Oja's rule:

<!-- Equation labels as ordinary links -->
<div id="eq:ojas_rule"></div>

$$
\begin{equation}
\tau_w \frac{d\mathbf{w}}{dt} = vu - \alpha v^2 w
\label{eq:ojas_rule} \tag{7}
\end{equation}
$$

Here, $\alpha$ is a positive constant. This rule only involves local synapse information, 
i.e., the pre- and postsynaptic activities and the local synaptic weight. 
However, its derivation is more theory-based than empirical-data-based. 
The enforced normalization is termed multiplicative because the second term's 
modification in Eq. ([7](#eq:ojas_rule)) is proportional to $w$.

The Oja rule's stability can be proven by computing the change of the weight vector length:

$$
\tau_w \frac{d|w|^2}{dt} = 2v^2 (1 - \alpha|w|^2)
$$

This shows that over time, $|w|^2$ converges to $\frac{1}{\alpha}$, 
effectively preventing the weights from growing indefinitely, hence ensuring stability. 
It also promotes weight competition because when one weight increases, the need to keep a 
constant weight vector length compels other weights to decrease.

## Exercise 3: Implement the Oja rule

**a)**

<!-- --- begin solution of exercise --- -->
**Solution.**

In [7]:
import numpy as np

def ojas_rule(x, w, alpha):
    """
    x: input vector
    w: weights vector
    alpha: learning rate
    """
    # calculate the dot product of x and w
    v = np.dot(w, x)
    
    # update the weights
    delta_w = alpha * v * (x - v * w)
    
    # return the updated weights
    return w + delta_w

# initialize weights
w = np.random.rand(10)

# define alpha
alpha = 0.01

# loop over some data (replace this with your actual data)
for _ in range(1000):
    # generate a random input
    x = np.random.rand(10)
    
    # update weights
    w = ojas_rule(x, w, alpha)

# print final weights
print(w)

This function takes in an input vector x and the current weights vector w, and it returns the 
updated weights vector after applying the Oja rule.

Note that the vector x and w should have the same dimensions, and alpha should be a small 
positive number (like 0.01). You might need to tune alpha based on your specific problem.

You would typically call this function in a loop, passing in your inputs and current weights, 
and updating your weights with the returned value.
<!-- --- end solution of exercise --- -->

**b)**
Show that the Oja rule converges to a solution similar to the PCA solution.

## Spike time dependent plasticity (STDP)
Spike-Timing Dependent Plasticity (STDP) is a fundamental biological process responsible for 
synaptic modification, thus playing a critical role in the learning and memory functions of 
the brain. In neuroscience, plasticity refers to the brain's ability to change and adapt in 
response to experience. This is accomplished by adjusting the strength of connections between 
neurons, which is modulated by their synaptic weights. 

STDP is a form of Hebbian plasticity that further refines the "fire together, wire together" 
principle by introducing an element of causality based on the precise timing of spikes. 
In STDP, the change in synaptic strength depends not just on the simultaneous firing of 
the pre- and post-synaptic neurons, but also on the order and timing of these firing events.

According to the rule of STDP, a synapse is strengthened if the presynaptic neuron fires 
just before the postsynaptic neuron (indicating that the presynaptic neuron might have 
contributed to the successful firing of the postsynaptic neuron). Conversely, if the 
presynaptic neuron fires just after the postsynaptic neuron, the synapse is weakened.

This form of synaptic plasticity takes into account the temporal relationship between the 
firings of the pre- and post-synaptic neurons, making it a more dynamic and precise form of 
Hebbian learning. It provides a mechanism for temporal coding, a neural coding scheme where 
information is encoded in the precise timings of spikes.

STDP has been observed in various types of neurons across many species, and has been shown to 
have significant effects on the learning and memory functions of neural circuits. 
Researchers continue to explore its implications for our understanding of the brain and 
for the development of neural network models in artificial intelligence.

## Exercise 4: Implement STDP in brian2

Exercise: Implementing Spike-Timing Dependent Plasticity (STDP) in Python

Implement a simple model of a spiking neural network with Spike-Timing Dependent Plasticity 
(STDP) using the Brian2 library in Python. Consider a model with the following characteristics:

1. There are 100 neurons in the network.
2. Each neuron obeys the simple leaky integrate-and-fire model with an input current and a decay time constant of 10 ms.
3. Neurons fire if their membrane potential exceeds 1 and reset to 0 after firing.
4. Each neuron is connected to every other neuron in the network through synapses that 
exhibit STDP. The synaptic weights should change according to the timing difference between 
the spikes of the pre- and postsynaptic neurons.

Implement the STDP model in Python using the Brian2 library and plot the synaptic weights, 
pre-synaptic activity, and post-synaptic activity as a function of time.

The model of the neuron is defined by the following differential equation:

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
\frac{{dv}}{{dt}} = \frac{{I-v}}{{10ms}}
\label{_auto2} \tag{8}
\end{equation}
$$

where $v$ is the membrane potential of the neuron and $I$ is the input current. 
Neurons fire if their membrane potential $v$ exceeds 1 and then $v$ is reset to 0.

The model of STDP is defined by the following set of equations:

$$
w : 1


\frac{{da_{pre}}}{{dt}} = -\frac{{a_{pre}}}{{\tau_{pre}}}


\frac{{da_{post}}}{{dt}} = -\frac{{a_{post}}}{{\tau_{post}}}
$$

where $w$ is the synaptic weight, $a_{pre}$ is the pre-synaptic activity, $a_{post}$ is the 
post-synaptic activity, $\tau_{pre}$ is the pre-synaptic time constant, and $\tau_{post}$ 
is the post-synaptic time constant. 

The STDP rule is implemented in the on_pre and on_post sections, where $w$ is incremented 
by $a_{post}$ after a pre-synaptic spike and by $a_{pre}$ after a post-synaptic spike.

Upon running the code, you should see the changes in synaptic weights, pre-synaptic 
activity, and post-synaptic activity over time.

In [8]:
from brian2 import *

# Parameters
taupre = taupost = 20*ms
Apre = 0.01
Apost = -Apre*taupre/taupost*1.05
tmax = 50*ms
N = 100

# Equations
eqs_neurons = '''
dv/dt = (I-v) / (10*ms) : 1 (unless refractory)
I : 1
'''
###### STDP rule code here #######

# Set up the monitors
mon = StateMonitor(S, ['w', 'apre', 'apost'], record=[0, 1])

# Run the simulation
run(tmax)

# Plot the results
subplot(311)
plot(mon.t/second, mon.w.T)
ylabel('w')
subplot(312)
plot(mon.t/second, mon.apre.T)
ylabel('apre')
subplot(313)
plot(mon.t/second, mon.apost.T)
xlabel('Time (s)')
ylabel('apost')
show()

<!-- --- begin solution of exercise --- -->
**Solution.**

In [9]:
from brian2 import *

# Parameters
taupre = taupost = 20*ms
Apre = 0.01
Apost = -Apre*taupre/taupost*1.05
tmax = 50*ms
N = 100
tau = 10*ms

# Equations
eqs_neurons = '''
dv/dt = (I-v) / tau : 1 (unless refractory)
I : 1
'''
eqs_synapses = '''
w : 1
dapre/dt = -apre/taupre : 1 (event-driven)
dapost/dt = -apost/taupost : 1 (event-driven)
'''
on_pre = '''
v_post += w
apre += Apre
w = clip(w+apost, 0, Inf)
'''
on_post = '''
apost += Apost
w = clip(w+apre, 0, Inf)
'''

# Set up the neurons
G = NeuronGroup(N, eqs_neurons, threshold='v>1', reset='v=0', refractory=5*ms)
G.I = linspace(1.51, 1.53, N)  # Different currents for different neurons

H = NeuronGroup(N, eqs_neurons, threshold='v>1', reset='v=0', refractory=5*ms)
H.I = linspace(1.51, 1.53, N)  # Different currents for different neurons
# Set up the synapses
S = Synapses(G, H, eqs_synapses,
             on_pre=on_pre,
             on_post=on_post)
S.connect(j='i')

# Set up the monitors
mon = StateMonitor(S, ['w', 'apre', 'apost'], record=[0, 1])
G_spikes = SpikeMonitor(G)
H_spikes = SpikeMonitor(H)

# Run the simulation
run(tmax)

# Plot the results
subplot(311)
plot(mon.t/second, mon.w.T)
ylabel('w')
subplot(312)
plot(mon.t/second, mon.apre.T)
ylabel('apre')
subplot(313)
plot(mon.t/second, mon.apost.T)
xlabel('Time (s)')
ylabel('apost')
show()

<!-- --- end solution of exercise --- -->